In [1]:
from langchain.embeddings import OpenAIEmbeddings

In [2]:
embeddings_model = OpenAIEmbeddings()

In [21]:
with open("q_and_a.txt", encoding='utf8') as f:
    real_estate_sales = f.read()

In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [29]:
# Should set chunk_size to 0, otherwise, CharacterTextSplitter will 
# capture more text into each object during splitting operation.
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.\n',
    chunk_size = 0,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

import re
def splitText(text):
    spliter = r'\d+\.\n'
    return re.split(spliter, text.strip())


In [ ]:
docs = text_splitter.create_documents([real_estate_sales])
# docs = list(filter(None,splitText(real_estate_sales)))

In [ ]:
print(len(docs))
[print(doc) for doc in docs]

In [34]:
## Load into FAISS

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())


In [40]:
query = "活动空间多嘛"

In [41]:
answer_list = db.similarity_search(query)

In [42]:
for ans in answer_list:
    print(ans.page_content + "\n")

[问题] 这个楼盘是否有社交活动场所，如社区中心或多功能厅？
[答案] 一些楼盘可能配备有社交活动场所，如社区中心、多功能厅或会议室，供居民举办各类社交活动、会议或聚会。您可以向楼盘的物业管理处了解相关设施情况。

[问题] 这个楼盘是否有游乐场或儿童活动区？
[答案] 一些楼盘可能提供游乐场或儿童活动区，为居民的孩子提供娱乐和活动场所。您可以向楼盘的物业管理处了解具体楼盘是否有游乐场或儿童活动区。

[问题] 这个楼盘是否有游乐场或儿童活动区？
[答案] 是的，楼盘会设置游乐场或儿童活动区，供业主的孩子们玩耍和娱乐。

[问题] 这个楼盘附近是否有运动场或健身设施？
[答案] 附近可能有运动场、健身房或其他健身设施，使居民可以方便地进行运动和健身活动。您可以通过查询当地的运动场所或健身设施信息来了解楼盘附近的情况。



In [43]:
## Save vector data to local db 
db.save_local("real_estates_sale")

In [44]:
#Top K retrieve 
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [45]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[问题] 这个楼盘是否有社交活动场所，如社区中心或多功能厅？
[答案] 一些楼盘可能配备有社交活动场所，如社区中心、多功能厅或会议室，供居民举办各类社交活动、会议或聚会。您可以向楼盘的物业管理处了解相关设施情况。

[问题] 这个楼盘是否有游乐场或儿童活动区？
[答案] 一些楼盘可能提供游乐场或儿童活动区，为居民的孩子提供娱乐和活动场所。您可以向楼盘的物业管理处了解具体楼盘是否有游乐场或儿童活动区。

[问题] 这个楼盘是否有游乐场或儿童活动区？
[答案] 是的，楼盘会设置游乐场或儿童活动区，供业主的孩子们玩耍和娱乐。



In [46]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")
for doc in docs:
    print(doc.page_content + "\n")

[问题] 这个楼盘是否有阳台或露台？
[答案] 大部分户型都配备有阳台或露台，让业主可以享受室外空间和阳光。

[问题] 这个楼盘是否有阳台或露台？
[答案] 许多楼盘都会提供阳台或露台，供居民享受户外空间。具体楼盘是否有阳台或露台可以向楼盘的销售团队或物业开发商咨询。

[问题] 这个楼盘的户型是多样化的吗？
[答案] 是的，楼盘提供多种户型选择，从80平方米至120平方米不等，包括一室一厅到三室两厅等不同户型。



In [51]:
#Use threshold 
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)


In [52]:
query = '有没有健身房'
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[问题] 这个楼盘是否有健身房或健身设施？
[答案] 一些楼盘可能配备有健身房或其他健身设施，供居民进行健身活动。您可以向楼盘的物业管理处了解具体楼盘是否提供健身房或健身设施。

[问题] 这个楼盘附近是否有运动场或健身设施？
[答案] 附近可能有运动场、健身房或其他健身设施，使居民可以方便地进行运动和健身活动。您可以通过查询当地的运动场所或健身设施信息来了解楼盘附近的情况。

[问题] 这个楼盘是否有游泳池或健身中心？
[答案] 一些楼盘可能配备有游泳池和健身中心等健身设施，供居民使用。您可以向楼盘的物业管理处了解具体楼盘是否提供游泳池或健身中心。



In [55]:
answer = docs[0].page_content.split('[答案]')

print(answer[-1].strip())

一些楼盘可能配备有健身房或其他健身设施，供居民进行健身活动。您可以向楼盘的物业管理处了解具体楼盘是否提供健身房或健身设施。


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [56]:
#create reused function 
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[答案] ")[-1] for doc in docs]

    return ans_list

In [66]:
#compare different threshold
testThreshold = 0.8
for i in range(4):
    steps = testThreshold - i * 0.03
    print(f'target threshold {steps}')
    result = sales('绿化多吗', steps)
    print(result)
    print('\n'*2)

target threshold 0.8
[]



target threshold 0.77
['小区绿化率高达35%，业主下楼就是花园，且有专业养护团队每天养护。', '楼盘有专业养护团队，每天负责绿化的维护工作，确保绿化环境的良好状态。']



target threshold 0.74
['小区绿化率高达35%，业主下楼就是花园，且有专业养护团队每天养护。', '楼盘有专业养护团队，每天负责绿化的维护工作，确保绿化环境的良好状态。', '附近可能有公园或绿地供居民休闲和娱乐。公园和绿地的数量和质量因地理位置而异。您可以查询当地的地理信息或向周边居民咨询了解附近是否有公园或绿地。', '楼盘附近可能有多个公园或绿地供居民休闲和户外活动。您可以查询当地的地理信息或向周边居民咨询，以了解楼盘附近的公园或绿地情况。']



target threshold 0.7100000000000001
['小区绿化率高达35%，业主下楼就是花园，且有专业养护团队每天养护。', '楼盘有专业养护团队，每天负责绿化的维护工作，确保绿化环境的良好状态。', '附近可能有公园或绿地供居民休闲和娱乐。公园和绿地的数量和质量因地理位置而异。您可以查询当地的地理信息或向周边居民咨询了解附近是否有公园或绿地。', '楼盘附近可能有多个公园或绿地供居民休闲和户外活动。您可以查询当地的地理信息或向周边居民咨询，以了解楼盘附近的公园或绿地情况。']





In [112]:
# go with LLM 
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate

NAME = '陈大文'
headerPrompt = f'你是专业的售楼顾问，名字叫{NAME}。'
bodyPrompt = f"""{headerPrompt} 
你需要按下述的'---'内容中每一个[答案]后的文字内容，总结后生成回复。
不要用'---'中的内容直接回复。
回复内容不需要出现你的名字，内容中不能出现'[答案]','[问题]','AI'等关键字。
如果'---'内容不存在，不要去生成任何不相关的回复, 只要回复‘我不知道’。"""
bodyPrompt = bodyPrompt + """\n
---
{context}
---
"""

template = PromptTemplate.from_template(bodyPrompt)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever = db.as_retriever(search_type="similarity_score_threshold",
                          search_kwargs={"score_threshold": 0.77, "k":3})

qa_chain = load_qa_chain(llm=llm, prompt=template, chain_type="stuff",verbose=True)
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever)

# qa = RetrievalQA.from_chain_type(llm, 
#                           retriever=retriever)

qa.combine_documents_chain.verbose = True
qa.return_source_documents = True

In [114]:
query = "小区有没有小学"
#qa({"query":query})
res = qa({"query":query})
# ans, sourcedocs = res['result'], res['source_documents']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是专业的售楼顾问，名字叫陈大文。 
你需要按下述的'---'内容中每一个[答案]后的文字内容，总结后生成回复。
不要用'---'中的内容直接回复。
回复内容不需要出现你的名字，内容中不能出现'[答案]','[问题]','AI'等关键字。
如果'---'内容不存在，不要去生成任何不相关的回复, 只要回复‘我不知道’。

---
[问题] 小区有幼儿园吗？
[答案] 小区有配套幼儿园，方便业主的孩子上学。

[问题] 这个楼盘附近有哪些教育机构？
[答案] 楼盘附近有配套的幼儿园、小学和初中，方便业主的子女接受教育。

[问题] 这个楼盘的周边是否有幼儿园或托儿所？
[答案] 附近可能有多个幼儿园或托儿所，为居民提供儿童教育和照顾服务。具体的幼儿园或托儿所可以通过查询当地的教育资源或向周边居民咨询了解。
---


> Finished chain.

> Finished chain.


In [115]:
ans, sourcedocs = res['result'], res['source_documents']
print(f'{ans},{1 if len(sourcedocs) >=1 else 0}')
print(res)

小区有配套幼儿园，方便业主的孩子上学。楼盘附近有配套的幼儿园、小学和初中，方便业主的子女接受教育。附近可能有多个幼儿园或托儿所，为居民提供儿童教育和照顾服务。具体的幼儿园或托儿所可以通过查询当地的教育资源或向周边居民咨询了解。,1
{'query': '小区有没有小学', 'result': '小区有配套幼儿园，方便业主的孩子上学。楼盘附近有配套的幼儿园、小学和初中，方便业主的子女接受教育。附近可能有多个幼儿园或托儿所，为居民提供儿童教育和照顾服务。具体的幼儿园或托儿所可以通过查询当地的教育资源或向周边居民咨询了解。', 'source_documents': [Document(page_content='[问题] 小区有幼儿园吗？\n[答案] 小区有配套幼儿园，方便业主的孩子上学。', metadata={}), Document(page_content='[问题] 这个楼盘附近有哪些教育机构？\n[答案] 楼盘附近有配套的幼儿园、小学和初中，方便业主的子女接受教育。', metadata={}), Document(page_content='[问题] 这个楼盘的周边是否有幼儿园或托儿所？\n[答案] 附近可能有多个幼儿园或托儿所，为居民提供儿童教育和照顾服务。具体的幼儿园或托儿所可以通过查询当地的教育资源或向周边居民咨询了解。', metadata={})]}
